In [33]:
!pip install requests beautifulsoup4 pandas python-dotenv


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [60]:
import time
from selenium import webdriver

driver = webdriver.Chrome()  # Optional argument, if not specified will search path.
driver.get('http://www.google.com/');
time.sleep(5) # Let the user actually see something!
search_box = driver.find_element_by_name('q')
search_box.send_keys('ChromeDriver')
search_box.submit()
time.sleep(5) # Let the user actually see something!
driver.quit()

AttributeError: 'WebDriver' object has no attribute 'find_element_by_name'

In [54]:
import requests
from bs4 import BeautifulSoup
import time

from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Access the username and password from the environment variables
username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')

# Create a session object to persist the login
session = requests.Session()

# URL of the login page
login_url = "https://www.marketscreener.com/login/?location=/"

# Replace with the actual form data required for login
payload = {
    'username': username,
    'password': password
}

# Headers (optional, but can help mimic browser behavior)
headers = {'User-Agent': 'Mozilla/5.0'}

# Submit login form
response = session.post(login_url, data=payload, headers=headers)

print(response.text)  # Print HTML content for debugging

# Check if login was successful
if response.status_code == 200:
    print("Login successful!")

def get_recommendation_urls_from_page(url, session):
    headers = {'User-Agent': 'Mozilla/5.0'}
    # Use session.get instead of requests.get to maintain the login state
    page = session.get(url, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')

    print(session.cookies.get_dict())


    # Find all recommendation links (adjust the HTML structure if necessary)
    links = soup.find_all('a', href=True)
    recommendation_urls = []
    
    for link in links:
        href = link['href']
        # Only include links with '/quote/stock/' and '/news/' but exclude '/news/hot-news/'
        if '/quote/stock/' in href and '/news/' in href:
            full_url = 'https://www.marketscreener.com' + href
            recommendation_urls.append(full_url)

    return recommendation_urls

# Function to loop through multiple pages with correct URL formatting
def get_all_recommendation_urls(base_url, cf_param, session, max_pages=10):
    all_recommendation_urls = []
    
    for p in range(1, max_pages + 1):
        # Correctly formatted URL with both p and cf parameters
        page_url = f"{base_url}?p={p}&cf={cf_param}"
        print(f"Scraping page {p}: {page_url}")
        # Pass session to the `get_recommendation_urls_from_page` function
        recommendation_urls = get_recommendation_urls_from_page(page_url, session)
        
        # If no URLs are found on this page, stop the loop
        if not recommendation_urls:
            print(f"No more recommendations found at page {p}. Stopping.")
            break
        
        all_recommendation_urls.extend(recommendation_urls)
        time.sleep(2)  # Be polite to the server by adding a small delay

    return all_recommendation_urls


<!DOCTYPE html><html lang="en-US" prefix="og: https://ogp.me/ns#"><head><meta charset="UTF-8"><title>Log In | MarketScreener</title><link rel='stylesheet' data-global=true href="https://cdn.zonebourse.com/static/compiled-css/global-v000054.min.css"><link data-version='v000054' rel='stylesheet' href="https://cdn.zonebourse.com"><link rel="stylesheet"
                  href="https://cdnjs.cloudflare.com/ajax/libs/bootstrap-icons/1.11.3/font/bootstrap-icons.min.css"
                  integrity="sha512-dPXYcDub/aeb08c63jRq/k6GaKccl256JQy/AnOq7CAnEZ9FzSL9wSbcZkMp4R26vBsMLFYH4kQ67/bbV8XaCQ=="
                  crossorigin="anonymous" referrerpolicy="no-referrer" /><link rel='preconnect' href='https://cdnjs.cloudflare.com/'><link rel='preconnect' href='https://securepubads.g.doubleclick.net/'><link rel='preconnect' href='https://www.googletagmanager.com/'><link rel='preconnect' href='https://client.px-cloud.net/'><link rel='preconnect' href='https://cdn.zonebourse.com/'><script>
    "use stri

In [51]:
# Example Usage of Part 1:
timeframe_cf_param = 'aVQwZTQzL1hkU0JOTloyNTNWTERkK0dzUlJtNWk0VjdHaHhrMk9LOXVSUkRpMXBLa3o2b0xhcHNzT3ZwdnF3VDhEbjhTelpKMlNMcitDNFNORE5aT2REL0NsZWlENytrK2EySElPUTh6U3R6MDhmdm10UkdPSUtmQldWTnhvVnY'

base_url = 'https://www.marketscreener.com/news/companies/recommandations/'
cf_param = timeframe_cf_param
# Pass the session object as an argument
all_recommendation_urls = get_all_recommendation_urls(base_url, cf_param, session, max_pages=10)

print(f"Total recommendation URLs found: {len(all_recommendation_urls)}")


Scraping page 1: https://www.marketscreener.com/news/companies/recommandations/?p=1&cf=aVQwZTQzL1hkU0JOTloyNTNWTERkK0dzUlJtNWk0VjdHaHhrMk9LOXVSUkRpMXBLa3o2b0xhcHNzT3ZwdnF3VDhEbjhTelpKMlNMcitDNFNORE5aT2REL0NsZWlENytrK2EySElPUTh6U3R6MDhmdm10UkdPSUtmQldWTnhvVnY
{'PHPSESSID': 'hdfcmdd2r0m6vigrd7vg5etklg', 'pv_r0_rand': '8', 'pv_r0_date': '2024-10-05', 'pv_r0': '1'}
Scraping page 2: https://www.marketscreener.com/news/companies/recommandations/?p=2&cf=aVQwZTQzL1hkU0JOTloyNTNWTERkK0dzUlJtNWk0VjdHaHhrMk9LOXVSUkRpMXBLa3o2b0xhcHNzT3ZwdnF3VDhEbjhTelpKMlNMcitDNFNORE5aT2REL0NsZWlENytrK2EySElPUTh6U3R6MDhmdm10UkdPSUtmQldWTnhvVnY
{'PHPSESSID': 'hdfcmdd2r0m6vigrd7vg5etklg', 'pv_r0_rand': '8', 'pv_r0_date': '2024-10-05', 'pv_r0': '2'}
Scraping page 3: https://www.marketscreener.com/news/companies/recommandations/?p=3&cf=aVQwZTQzL1hkU0JOTloyNTNWTERkK0dzUlJtNWk0VjdHaHhrMk9LOXVSUkRpMXBLa3o2b0xhcHNzT3ZwdnF3VDhEbjhTelpKMlNMcitDNFNORE5aT2REL0NsZWlENytrK2EySElPUTh6U3R6MDhmdm10UkdPSUtmQldWTnhvVnY
{'PHPSESSID':

In [55]:
# Assuming you've already set up the session after logging in
# The session object should persist across all requests

def scrape_recommendation_page(url, session):
    headers = {'User-Agent': 'Mozilla/5.0'}

    print(session.cookies.get_dict())
    try:
        # Use session.get instead of requests.get to maintain the login state
        page = session.get(url, headers=headers)
        soup = BeautifulSoup(page.content, 'html.parser')
    except Exception as e:
        print(f"Error fetching page {url}: {e}")
        return None
    
    # Extract the published date
    date_div = soup.find('div', class_='c-6 mb-15')
    published_date = date_div.get_text().strip() if date_div else 'No Date'
    
    # Check if the published date is in "September ... 2024"
    if re.search(r'September.*2024', published_date) or re.search(r'\d{2}\.09\.2024', published_date) or re.search(r'\d{2}\.10\.2024', published_date) or re.search(r'October.*2024', published_date):
        print(f"Skipping page {url}: Published in September 2024")
        return None  # Skip this page

    # Extract the title
    title_tag = soup.find('h1', class_='title title__primary mb-15 txt-bold')
    title = title_tag.get_text().strip() if title_tag else 'No Title'

    # Extract the full text
    full_text_div = soup.find('div', class_='txt-s4 article-text')
    full_text = full_text_div.get_text().strip() if full_text_div else 'No Content'

    # Skip if the title or full text is 'No Content'
    if  full_text == 'No Content':
        print(f"Skipping page {url}: Missing content")
        return None

    # Extract the source
    source_div = soup.find('div', class_='c-auto mb-15 txt-align-right txt-s2')
    source = source_div.get_text().strip() if source_div else 'No Source'

    # Extract the company name
    company_name_header = soup.find('h2', class_='m-0 txt-s1 txt-b5')
    company_name = company_name_header.get_text().strip() if company_name_header else 'No Company Name'

    additional_infos = soup.find_all('div', class_='c-auto txt-align-right txt-bold')
    if additional_infos:
        additional_infos_all = [info.get_text().strip() for info in additional_infos]
        mean_consensus = additional_infos_all[0] if len(additional_infos_all[0]) > 0 else 'No Mean Consensus'
        number_of_analysts = additional_infos_all[1] if len(additional_infos_all[1]) > 0 else 'No Number of Analysts'
        last_closed_price = additional_infos_all[2] if len(additional_infos_all[2]) > 0 else 'No Last Closed price'
        average_target_price = additional_infos_all[3] if len(additional_infos_all[3]) > 0 else 'No Average Target Price'
        spread = additional_infos_all[4] if len(additional_infos_all[4]) > 0 else 'No Spread'

    # Extract company information (handle case where not found)
    company_information_badges = soup.find_all('h2', class_='m-0 badge txt-b5 txt-s1')
    if company_information_badges:
        company_information = [badge.get_text().strip() for badge in company_information_badges]
        company_name_short = company_information[0] if len(company_information) > 0 else 'No Content'
        company_id = company_information[1] if len(company_information) > 1 else 'No Content'
    else:
        company_name_short = 'No Content'
        company_id = 'No Content'

    # Extract industry information (handle case where not found)
    industry_badges = soup.find_all('h2', class_='m-0 txt-b5 txt-s1')
    if industry_badges:
        industry_information = [badge.get_text().strip() for badge in industry_badges]
        industry_general = industry_information[0] if len(industry_information) > 0 else 'No Industry General'
        industry = industry_information[1] if len(industry_information) > 1 else 'No industry tag'
    else:
        industry_general = 'No Industry General'
        industry = 'No industry tag'
    
    # Return the results as a dictionary
    return {
        'url': url,
        'title': title,
        'published_date': published_date,
        'full_text': full_text,
        'source': source,
        'company_name': company_name,
        'company_name_short': company_name_short,
        'company_id': company_id,
        'industry_general': industry_general,
        'industry': industry,
        'mean_consensus': mean_consensus,
        'number_of_analysts': number_of_analysts,
        'last_closed_price': last_closed_price,
        'average_target_price': average_target_price,
        'spread': spread
    }

# Modify this function to pass the session as well
def scrape_all_recommendations(urls, session):
    data = []
    print(session.cookies.get_dict())
    
    for url in urls:
        try:
            recommendation_data = scrape_recommendation_page(url, session)
            if recommendation_data:  # Only add if not None (i.e., not skipped)
                data.append(recommendation_data)
                print(f"Scraped: {recommendation_data['title']}")
        except Exception as e:
            print(f"Error scraping page {url}: {e}")
            time.sleep(2)
    
    return data


In [56]:
# Example Usage of Part 3:
recommendation_data = scrape_all_recommendations(all_recommendation_urls,session)

# Convert the data into a DataFrame
df = pd.DataFrame(recommendation_data)

{'PHPSESSID': '5ulfu1fi9rfvhii8646tmedvob'}
{'PHPSESSID': '5ulfu1fi9rfvhii8646tmedvob'}
Skipping page https://www.marketscreener.com/quote/stock/APOGEE-ENTERPRISES-INC-8400/news/Apogee-Enterprises-Inc-Reports-Earnings-Results-for-the-Second-Quarter-and-Six-Months-Ended-August-48002209/: Published in September 2024
{'PHPSESSID': '5ulfu1fi9rfvhii8646tmedvob', 'pv_r0_rand': '10', 'pv_r0_date': '2024-10-05', 'pv_r0': '1'}
Skipping page https://www.marketscreener.com/quote/stock/SPIRIT-AIRLINES-INC-39143167/news/Spirit-Airlines-shares-plunge-on-report-of-potential-bankruptcy-filing-48001261/: Published in September 2024
{'PHPSESSID': '5ulfu1fi9rfvhii8646tmedvob', 'pv_r0_rand': '10', 'pv_r0_date': '2024-10-05', 'pv_r0': '2'}
Skipping page https://www.marketscreener.com/quote/stock/BARNES-GROUP-INC-11749/news/Exclusive-Apollo-Global-in-talks-to-buy-aerospace-parts-maker-Barnes-Group-sources-say-47997312/: Published in September 2024
{'PHPSESSID': '5ulfu1fi9rfvhii8646tmedvob', 'pv_r0_rand': '1

In [57]:
# Save the DataFrame to a CSV file
df.to_csv('/Users/oskarroeske/Masterthesis/scraped_data_us_market/analyst_recommendations_login_test2.csv', index=False)
print("Data saved")

Data saved
